In [1]:
import requests 
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import tqdm

In [4]:
def jtbc_crawling(start_date, end_date):
    url1 = []
    # url 변화하는거 적당히 보고 알아서 잘 해야됨. 페이지 단위로 넘어가는게 아니라면 selenium 써야할 수도 있음.
    url_head = "http://jtbc.joins.com/search/news?page="
    url_tail = "&term=kaist"
    for page in tqdm(range(1,19)):
        url_final = url_head + str(page) + url_tail
        response = requests.get(url_final)
        html = response.content
        soup = bs(html, "html.parser")
        elements = soup.select('.prg_ttl a[href]') # F12 눌러서 적당히 우리가 원하는 a href 태그 url 링크 달려있는 id 또는 class 찾아서 넣어주면 됨. 
        for element in elements:
            link = element.attrs['href']
            url1.append(link)
    url2 = []
    # url 변화하는거 적당히 보고 알아서 잘 해야됨. 페이지 단위로 넘어가는게 아니라면 selenium 써야할 수도 있음.
    url_head = "http://jtbc.joins.com/search/news?page="
    url_tail = "&term=카이스트"
    for page in tqdm(range(1,100)):
        url_final = url_head + str(page) + url_tail
        response = requests.get(url_final)
        html = response.content
        soup = bs(html, "html.parser")
        elements = soup.select('.prg_ttl a[href]') # F12 눌러서 적당히 우리가 원하는 a href 태그 url 링크 달려있는 id 또는 class 찾아서 넣어주면 됨. 
        for element in elements:
            link = element.attrs['href']
            url2.append(link)
    url = list(set(url1).union(set(url2)))
    df = pd.DataFrame({"url": url}, columns=["url"])
    contents = []
    titles = []
    urls = []
    for row in tqdm(df.itertuples()):
        link = getattr(row, 'url')
        response = requests.get(link)
        html = response.text
        soup = bs(html,'html.parser')
        try:
            for script in soup(["script", "style"]):
                script.decompose() 
            date = soup.select('.i_date')[0].get_text()
            date = date.split(' ')[1]
            content = soup.select('#articlebody')[0].get_text() # select 안에 본문에 해당하는 id 값을 찾아서 넣어주면 됨.
            content = content.replace('\xa0', " ").replace('\u3000','').replace('\r','').replace('\n', " ")
            contents.append(content)
            title = soup.select('#jtbcBody h3')[0].get_text()
            title = title.replace('\xa0', " ")
            title = title.replace('\n', " ")
            titles.append(title)
            urls.append(link)
        except:
            print(row)
    df = pd.DataFrame({"url": urls, "title": titles, "content": contents}, columns=["url", "title", "content"])
    df.to_excel("articles/jtbc.xlsx",index = False)
    df.to_pickle("pickle files/jtbc.pkl")

In [5]:
jtbc_crawling('2015.01.01', '2020.10.23')

100%|██████████████████████████████████████████████████████████████████████████████████| 99/99 [00:33<00:00,  2.92it/s]
1110it [04:00,  4.62it/s]
